In [1]:
%%capture
!python -m spacy download ru_core_news_lg 
!pip install natasha pandas numpy scikit-learn nltk
# !pip install catboost

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch

import pandas as pd
import numpy as np
import re
import random 
import spacy
from spacy.training.example import Example
from tqdm import tqdm
import os
import json
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_curve, f1_score
from sklearn.neighbors import KNeighborsClassifier

import gc # garbage collect library
import torch 

import tensorflow as tf
# import autokeras as ak
from nltk.corpus import stopwords

# import mlflow
# from mlflow.models.signature import infer_signature
# from mlflow.tracking import MlflowClient

import natasha
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
   
    PER,
    Doc
)
from sklearn.utils import resample
from itertools import combinations, combinations_with_replacement, permutations

import pickle 
# from catboost import CatBoostClassifier, Pool, cv, EShapCalcType, EFeaturesSelectionAlgorithm

2024-04-27 18:25:35.506593: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 18:25:35.506648: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 18:25:35.508253: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
data_train = pd.read_csv("/kaggle/input/geek-bot-dataset/train_Feedback/train_data.csv")
data_train.head()

,timestamp,question_1,question_2,question_3,question_4,question_5,is_relevant,object,is_positive
0,2024-04-01 08:30:00,Основы программирования,Интерактивный подход к изучению основ C# был о...,Нет.,Увеличить количество практических заданий для ...,Введение в разработку на .NET Framework.,1,0,1
1,2024-04-01 10:00:00,Основы программирования,Подробное объяснение логических операторов в J...,Нет.,Предоставить ещё больше практических задач пос...,Детальное изучение асинхронного программирован...,1,0,1
2,2024-04-01 15:00:00,Основы программирования,Введение в программирование на Python было про...,Нет.,"Улучшить качество видеоматериалов, иногда было...",Библиотеки Python для анализа данных и машинно...,1,0,1
3,2024-04-01 17:00:00,Основы программирования,"Понятное изложение основных понятий, хорошая с...","Да, циклы for и while были объяснены слишком б...",Добавить видеоматериалы для самостоятельного и...,Функциональное программирование.,1,0,1
4,2024-04-02 09:00:00,Основы программирования,Кларность и структура материала по основам Jav...,Нет.,Ввести больше примеров на работу с массивами и...,Работа с асинхронным кодом в JavaScript.,1,0,1


## Скрипты для лемантизации и очистки текста

In [6]:
#Load AutoModel from huggingface model repository
device = "cuda"
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_mt_nlu_ru", device_map = 'cuda')
model = AutoModel.from_pretrained("ai-forever/sbert_large_mt_nlu_ru", device_map = 'cuda')

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [78]:
def augmentation_data(data: pd.DataFrame, label: str = 'is_relevant'):
    augment_data = pd.DataFrame()

    col_number = [2, 3, 4, 5]

    # Генерация всех возможных комбинаций без повторений
    for len_comb in range(2, 5):
        # Комбинация колонок
        comb_list = list(permutations(col_number, len_comb))
        

        # Переберем все комбинации
        for comb in comb_list:

            data_buffer = pd.DataFrame()
            data_buffer["text"] = data[f'question_{comb[0]}'].copy()

            for col in comb[1:]:
                data_buffer['text'] = data_buffer['text'] + " " + data[f'question_{col}'].copy()
            
            data_buffer["label"] = data[label]
            
            if label not in ["object", "is_positive"]: # "is_positive"
                # data_buffer = data_buffer[data_buffer["label"] == 0]
                data_buffer_neg = data_buffer[data_buffer["label"] == 0].copy()
                
                data_buffer_pos = resample(data_buffer[data_buffer["label"] == 1],
                     replace=False,
                     n_samples = data_buffer_neg.shape[0],
                     random_state=12345)
                
                data_buffer = pd.concat([data_buffer_neg, data_buffer_pos], ignore_index=True)
                # print(data_buffer["label"].value_counts())
            
            augment_data = pd.concat([augment_data, data_buffer], ignore_index=True)
    
    print(augment_data['label'].value_counts())
    return augment_data


def natasha_lemmant(input_str, segmenter, morph_tagger, syntax_parser,morph_vocab):
    doc = Doc(input_str)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)

    result = ' '.join([_.lemma for _ in doc.tokens])
    return result


def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


def clear_string(s, stop_words): # Функция для очистки
    # print(re.sub(r'[^а-яА-ЯёЁa-zA-Z]', ' ', str(s).lower()).split())
    s = ' '.join([val for val in re.sub(r'[^а-яА-ЯёЁa-zA-Z]', ' ', str(s).lower()).split() if not val in stop_words])
    # s = ' '.join([val for val in re.sub(r'[^а-яА-ЯёЁ]', ' ', str(s).lower()).split() if not val in stop_words])
    return s


def create_dataset(data, label, is_augment = False):
    stop_words = list(stopwords.words('russian'))
    
    segmenter = Segmenter()
    morph_vocab = MorphVocab()
    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    syntax_parser = NewsSyntaxParser(emb)
    
    if is_augment == True:
        data = augmentation_data(data, label)
    
    else:
        data['text'] = data['question_2'] + " " + data['question_3'] + " " + data['question_4'] + " " + data['question_5']
        data["label"] = data[label]
    
    data["text"] = data["text"].apply(lambda x: natasha_lemmant(clear_string(x, stop_words), segmenter, morph_tagger, syntax_parser, morph_vocab))
    
    
    #Tokenize sentences
    # print(data["text"].to_numpy())
    
    sentence_embeddings = None
    
    text_data = data["text"].to_list()
    
    batch = 512
    
    for i in tqdm(range(0, data.shape[0], batch), desc = "Batch"):
        encoded_input = tokenizer(text_data[i:i+batch], padding=True, truncation=True, max_length = 1024, return_tensors='pt').to(device)
        with torch.no_grad():
            #Compute token embeddings
            model_output = model(**encoded_input)

        #Perform pooling. In this case, mean pooling
        # print(encoded_input['attention_mask'])
        # print(model_output)
        embeddings = mean_pooling(model_output, encoded_input['attention_mask']).cpu().numpy()
        if i > 0:
            sentence_embeddings = np.append(sentence_embeddings, embeddings, axis = 0)
        else:
            sentence_embeddings = embeddings
            # print(embeddings_list)
    
    gc.collect()
    torch.cuda.empty_cache() 
    print(sentence_embeddings.shape)
    return sentence_embeddings, data["label"].to_numpy()

## Разделяем данные на тренировочную и валидационную выборку

In [79]:
data_relevant_train, data_relevant_val = train_test_split(
    data_train,
    test_size=0.33,
    random_state=12345,
    stratify = data_train['is_relevant']
)

data_relevant_train["is_relevant"].value_counts(), data_relevant_val["is_relevant"].value_counts()

(is_relevant
 1    60
 0     9
 Name: count, dtype: int64,
 is_relevant
 1    30
 0     4
 Name: count, dtype: int64)

In [80]:
data_object_train, data_object_val = train_test_split(
    data_train,
    test_size=0.33,
    random_state=12345,
    stratify = data_train['object']
)

data_object_train["object"].value_counts(), data_object_val["object"].value_counts()

(object
 0    26
 2    23
 1    20
 Name: count, dtype: int64,
 object
 2    12
 0    12
 1    10
 Name: count, dtype: int64)

In [81]:
data_positive_train, data_positive_val = train_test_split(
    data_train,
    test_size=0.33,
    random_state=12345,
    stratify = data_train['is_positive']
)

data_positive_train["is_positive"].value_counts(), data_positive_val["is_positive"].value_counts()

(is_positive
 1    56
 0    13
 Name: count, dtype: int64,
 is_positive
 1    27
 0     7
 Name: count, dtype: int64)

## Очищаем данные и создаём эмбединги

In [82]:
%%time
data_relevant_train_emb, data_relevant_train_lbl  = create_dataset(data_relevant_train, label = "is_relevant", is_augment = True)
data_relevant_val_emb, data_relevant_val_lbl = create_dataset(data_relevant_val, label = "is_relevant")

data_object_train_emb, data_object_train_lbl = create_dataset(data_object_train, label = "object", is_augment = True)
data_object_val_emb, data_object_val_lbl = create_dataset(data_object_val, label = "object")

data_positive_train_emb, data_positive_train_lbl = create_dataset(data_positive_train, label = "is_positive", is_augment = True)
data_positive_val_emb, data_positive_val_lbl = create_dataset(data_positive_val, label = "is_positive")

label
0    540
1    540
Name: count, dtype: int64


Batch: 100%|██████████| 3/3 [00:04<00:00,  1.64s/it]


(1080, 1024)


Batch: 100%|██████████| 1/1 [00:00<00:00,  5.18it/s]


(34, 1024)
label
0    1560
2    1380
1    1200
Name: count, dtype: int64


...

(34, 1024)
label
1    3360
0     780
Name: count, dtype: int64


Batch: 100%|██████████| 9/9 [00:19<00:00,  2.20s/it]


(4140, 1024)


Batch: 100%|██████████| 1/1 [00:00<00:00,  2.31it/s]


(34, 1024)
CPU times: user 4min 5s, sys: 2min 1s, total: 6min 7s
Wall time: 2min 37s


In [83]:
data_relevant_train_emb.shape, data_relevant_train_lbl.shape

((1080, 1024), (1080,))

In [73]:
# model = CatBoostClassifier(
#     verbose = 100,
#     learning_rate=0.001,
#     # eval_metric='Logloss',
#     iterations = 2000,
#     random_seed = 12345,
#      # task_type="GPU",
    
# )

# pool_train = Pool(
#     data_relevant_train_emb,
#     label= data_relevant_train_lbl, 
# )

# pool_val = Pool(
#     data_relevant_val_emb,
#     label= data_relevant_val_lbl,
# )

# model.fit(pool_train, eval_set = pool_val)

# print(model.get_all_params())

In [74]:
# f1_score(data_relevant_val_lbl, model.predict(data_relevant_val_emb))

## Модель relevant

In [84]:
best_metrics_relevant = 0
best_k = 0

for k in range(3, 15):
    knn = KNeighborsClassifier(n_neighbors=k, n_jobs = -1, weights = "uniform") #  metric = LogLoss

    knn.fit(data_relevant_train_emb, data_relevant_train_lbl)

    pred_val = knn.predict(data_relevant_val_emb)
    
    metric = f1_score(data_relevant_val_lbl, pred_val)
    
    if metric > best_metrics_relevant:
        best_k = k
        best_metrics_relevant = metric
        
        with open("/kaggle/working/best_model_relevant.pkl", "wb") as f:
            pickle.dump(knn, f)
        
    print(f'k = {k}\tf1_score: {metric}')
    
print('\n\n', '*' * 100)
print(f"best_k: {best_k}\tbest_f1: {best_metrics_relevant}")

k = 3	f1_score: 0.967741935483871
k = 4	f1_score: 0.967741935483871
k = 5	f1_score: 0.967741935483871
k = 6	f1_score: 0.967741935483871
k = 7	f1_score: 0.967741935483871
k = 8	f1_score: 0.9836065573770492
k = 9	f1_score: 0.967741935483871
k = 10	f1_score: 0.967741935483871
k = 11	f1_score: 0.967741935483871
k = 12	f1_score: 0.9836065573770492
k = 13	f1_score: 0.967741935483871
k = 14	f1_score: 0.967741935483871


 ****************************************************************************************************
best_k: 8	best_f1: 0.9836065573770492


## Модель object

In [85]:
best_metrics_object = 0
best_k = 0

for k in range(3, 15):
    knn = KNeighborsClassifier(n_neighbors = k, n_jobs = -1, weights = "uniform") #  metric = LogLoss

    knn.fit(data_object_train_emb, data_object_train_lbl)

    pred_val = knn.predict(data_object_val_emb)
    # print(data_object_val_lbl)
    # print(pred_val)
    
    metric = f1_score(data_object_val_lbl, pred_val, average = 'micro')
    
    if metric > best_metrics_object:
        best_k = k
        best_metrics_object = metric
        
        with open("/kaggle/working/best_model_object.pkl", "wb") as f:
            pickle.dump(knn, f)
        
    print(f'k = {k}\tf1_score: {metric}')

print('\n\n', '*' * 100)
print(f"best_k: {best_k}\tbest_f1: {best_metrics_object}")

k = 3	f1_score: 0.7058823529411765
k = 4	f1_score: 0.6176470588235294
k = 5	f1_score: 0.6176470588235294
k = 6	f1_score: 0.6176470588235294
k = 7	f1_score: 0.6470588235294118
k = 8	f1_score: 0.7058823529411765
k = 9	f1_score: 0.735294117647059
k = 10	f1_score: 0.7058823529411765
k = 11	f1_score: 0.7058823529411765
k = 12	f1_score: 0.6764705882352942
k = 13	f1_score: 0.6764705882352942
k = 14	f1_score: 0.6470588235294118


 ****************************************************************************************************
best_k: 9	best_f1: 0.735294117647059


## Модель is_positive

In [86]:
best_metrics_positive = 0
best_k = 0

for k in range(3, 15):
    knn = KNeighborsClassifier(n_neighbors = k, n_jobs = -1, weights = "uniform") #  metric = LogLoss

    knn.fit(data_positive_train_emb, data_positive_train_lbl)

    pred_val = knn.predict(data_positive_val_emb)
    # print(data_object_val_lbl)
    # print(pred_val)
    
    metric = f1_score(data_positive_val_lbl, pred_val)
    
    if metric > best_metrics_positive:
        best_k = k
        best_metrics_positive = metric
        
        with open("/kaggle/working/best_model_positive.pkl", "wb") as f:
            pickle.dump(knn, f)
        
    print(f'k = {k}\tf1_score: {metric}')

print('\n\n', '*' * 100)
print(f"best_k: {best_k}\tbest_f1: {best_metrics_positive}")

k = 3	f1_score: 0.912280701754386
k = 4	f1_score: 0.912280701754386
k = 5	f1_score: 0.9310344827586207
k = 6	f1_score: 0.9310344827586207
k = 7	f1_score: 0.9310344827586207
k = 8	f1_score: 0.9473684210526316
k = 9	f1_score: 0.9473684210526316
k = 10	f1_score: 0.9473684210526316
k = 11	f1_score: 0.9473684210526316
k = 12	f1_score: 0.9473684210526316
k = 13	f1_score: 0.9310344827586207
k = 14	f1_score: 0.9473684210526316


 ****************************************************************************************************
best_k: 8	best_f1: 0.9473684210526316


## Итоговая метрика

In [87]:
(best_metrics_relevant + best_metrics_positive + best_metrics_object) / 3

0.8887563653589132

In [21]:
#Sentences we want sentence embeddings for
# sentences = ['тема понравиться обсуждаться распространить среди аналитик инструмент colab такой момент раскрыть следующий тема такой колаб описание интерфейс правило работа колаб нужный функция колабый библиотека колабнуть открыть колаб вижуал студио установить модуль открытие колабый вижуал студио это первый прийти ум кстати зря прислать ссылка урок урок комментарий недовольный пользователь рассказать рассказать предыдущий вопрос такой колаб описание интерфейс правило работа колаб нужный функция колабый библиотека колабнуть открыть колаб вижуал студио установить модуль открытие колабый вижуал студио библиотека колаб']

In [44]:
# #Tokenize sentences
# encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length = 1024, return_tensors='pt')

# #Compute token embeddings
# with torch.no_grad():
#     model_output = model(**encoded_input)
    
# #Perform pooling. In this case, mean pooling
# sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
# sentence_embeddings.shape

torch.Size([1, 1024])

In [46]:
# sentence_embeddings.cpu().numpy()

array([[0.6765646 , 0.204501  , 0.20930381, ..., 0.05056305, 0.62336695,
        0.03852016]], dtype=float32)